In [2]:
from transformers import Trainer, BertTokenizer, BertForSequenceClassification
from datasets import load_from_disk
import optuna
import torch
import math
import base

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/jovyan/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package punkt to /home/jovyan/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to /home/jovyan/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger_eng to
[nltk_data]     /home/jovyan/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger_eng is already up-to-
[nltk_data]       date!


In [3]:
if torch.cuda.is_available():
    device = torch.device("cuda")
    print("GPU is available and will be used:", torch.cuda.get_device_name(0))
else:
    device = torch.device("cpu")
    print("GPU is not available, using CPU.")

GPU is available and will be used: NVIDIA A100 80GB PCIe MIG 1g.10gb


In [4]:
DATASET = "trec"

In [5]:
train_data = load_from_disk(f"~/data/{DATASET}/train-logits_coarse")
eval_data = load_from_disk(f"~/data/{DATASET}/eval-logits_coarse")
test_data = load_from_disk(f"~/data/{DATASET}/test-logits_coarse")

all_train_data = load_from_disk(f"~/data/{DATASET}/train-logits-augmented_coarse")
tokenizer = BertTokenizer.from_pretrained("ndavid/autotrain-trec-fine-bert-739422530")

In [6]:
train = train_data.map(lambda e: tokenizer(e["sentence"], truncation=True, padding="max_length", return_tensors="pt", max_length=300), batched=True, desc="Tokenizing the train dataset")
eval = eval_data.map(lambda e: tokenizer(e["sentence"], truncation=True, padding="max_length", return_tensors="pt", max_length=300), batched=True, desc="Tokenizing the eval dataset")
test = test_data.map(lambda e: tokenizer(e["sentence"], truncation=True, padding="max_length", return_tensors="pt", max_length=300), batched=True, desc="Tokenizing the test dataset")

train_aug = all_train_data.map(lambda e: tokenizer(e["sentence"], truncation=True, padding="max_length", return_tensors="pt", max_length=300), batched=True, desc="Tokenizing the augmented dataset")

In [7]:
num_epochs = 15
batch_size = 128

In [8]:
#Nápočet epoch na steps
data_length = len(train_data)
min_r = math.ceil(data_length/batch_size)*5
max_r = math.ceil(data_length/batch_size)*num_epochs
warm_up = math.ceil(data_length/batch_size/10)

In [9]:
def hp_space(trial):
    params =  {
        "learning_rate": trial.suggest_float("learning_rate", 1e-5, 5e-4, log=True),
        "weight_decay": trial.suggest_float("weight_decay", 0, 1e-2, step=1e-3),
        "warmup_steps" : trial.suggest_int("warmup_steps", 0, warm_up)
    }   
    print(f"Trial {trial.number} with params: {params}")
    return params

In [10]:
pruner = optuna.pruners.HyperbandPruner(min_resource=min_r, max_resource=max_r, reduction_factor=2, bootstrap_count=2)
sampler = optuna.samplers.TPESampler(seed=42, multivariate=True)

/usr/local/lib/python3.10/dist-packages/optuna/_experimental.py:31: ExperimentalWarning: Argument ``multivariate`` is an experimental feature. The interface can change in the future.
  warnings.warn(


In [11]:
def get_Bert():
    return BertForSequenceClassification.from_pretrained("google/bert_uncased_L-2_H-128_A-2", num_labels=6)

In [12]:
base.reset_seed()

In [12]:
training_args = base.get_training_args(output_dir=f"~/results/{DATASET}/bert-base_coarse_hp-search", logging_dir=f"~/logs/{DATASET}/bert-base_coarse_hp-search", epochs=num_epochs, batch_size=batch_size)

In [13]:
trainer = Trainer(
    args=training_args,
    train_dataset=train,
    eval_dataset=eval,
    compute_metrics=base.compute_metrics,
    model_init = lambda: get_Bert(),
)
  

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at google/bert_uncased_L-2_H-128_A-2 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [14]:
best_trial = trainer.hyperparameter_search(
    direction="maximize",
    backend="optuna",
    hp_space=hp_space,
    compute_objective=lambda metrics: metrics["eval_f1"],
    pruner=pruner,
    sampler=sampler,
    study_name="Test-base",
    n_trials=150
)

[I 2025-03-27 19:01:11,849] A new study created in memory with name: Test-base


Trial 0 with params: {'learning_rate': 4.3284502212938785e-05, 'weight_decay': 0.01, 'warmup_steps': 3}


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at google/bert_uncased_L-2_H-128_A-2 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,1.750700,1.682168,0.289643,0.180136,0.215374,0.174118
2,1.661400,1.604126,0.413382,0.203241,0.312456,0.235986
3,1.584200,1.504832,0.451879,0.363096,0.348685,0.291920
4,1.482600,1.398842,0.547204,0.527174,0.444096,0.427751
5,1.380400,1.299083,0.626031,0.574708,0.525069,0.524653
6,1.284900,1.213912,0.681943,0.604434,0.580901,0.579261
7,1.201500,1.141451,0.705775,0.617201,0.602667,0.600785
8,1.134100,1.079239,0.721357,0.624162,0.617420,0.614636
9,1.073200,1.033059,0.732356,0.633858,0.626941,0.623792
10,1.028500,0.994233,0.744271,0.639393,0.637056,0.633078


[I 2025-03-27 19:02:01,862] Trial 0 pruned. 


...

[I 2025-03-27 21:51:45,732] Trial 148 finished with value: 0.8491745793974798 and parameters: {'learning_rate': 0.000427837974552112, 'weight_decay': 0.008, 'warmup_steps': 2}. Best is trial 87 with value: 0.8633510888244157.


Trial 149 with params: {'learning_rate': 0.00023331250665160932, 'weight_decay': 0.003, 'warmup_steps': 2}


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at google/bert_uncased_L-2_H-128_A-2 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,1.618100,1.363934,0.562786,0.564468,0.451293,0.424837
2,1.160000,0.896022,0.768103,0.660638,0.656149,0.656161
3,0.745600,0.632110,0.834097,0.705496,0.712378,0.708520
4,0.520100,0.527115,0.852429,0.721305,0.725193,0.722462
5,0.395900,0.469241,0.867094,0.728754,0.740810,0.734452


[I 2025-03-27 21:52:16,015] Trial 149 pruned. 


In [15]:
print(best_trial)

BestRun(run_id='87', objective=0.8633510888244157, hyperparameters={'learning_rate': 0.00046314539247556066, 'weight_decay': 0.001, 'warmup_steps': 3}, run_summary=None)


In [16]:
base.reset_seed()

In [17]:
training_args = base.get_training_args(output_dir=f"~/results/{DATASET}/bert-distill_coarse_hp-search", logging_dir=f"~/logs/{DATASET}/bert-distill_coarse_hp-search", remove_unused_columns=False, epochs=num_epochs, batch_size=batch_size)

In [18]:
def hp_space(trial):
    params =  {
        "learning_rate": trial.suggest_float("learning_rate", 1e-5, 5e-4, log=True),
        "weight_decay": trial.suggest_float("weight_decay", 0, 1e-2, step=1e-3),
        "warmup_steps" : trial.suggest_int("warmup_steps", 0, warm_up),
        "lambda_param": trial.suggest_float("lambda_param",0,1,step=.1),
        "temperature": trial.suggest_float("temperature", 2,7, step=.5)
    }
    print(f"Trial {trial.number} with params: {params}")
    return params

In [19]:
pruner = optuna.pruners.HyperbandPruner(min_resource=min_r, max_resource=max_r, reduction_factor=2, bootstrap_count=2)
sampler = optuna.samplers.TPESampler(seed=42, multivariate=True)

/usr/local/lib/python3.10/dist-packages/optuna/_experimental.py:31: ExperimentalWarning: Argument ``multivariate`` is an experimental feature. The interface can change in the future.
  warnings.warn(


In [20]:
trainer = base.DistilTrainer(
    args=training_args,
    train_dataset=train,
    eval_dataset=eval,
    compute_metrics=base.compute_metrics,
    model_init = lambda: get_Bert(),
)
  

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at google/bert_uncased_L-2_H-128_A-2 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [21]:
best_trial2 = trainer.hyperparameter_search(
    direction="maximize",
    backend="optuna",
    hp_space=hp_space,
    compute_objective=lambda metrics: metrics["eval_f1"],
    pruner=pruner,
    sampler=sampler,
    study_name="Distilation",
    n_trials=150
)

[I 2025-03-27 21:52:16,565] A new study created in memory with name: Distilation


Trial 0 with params: {'learning_rate': 4.3284502212938785e-05, 'weight_decay': 0.01, 'warmup_steps': 3, 'lambda_param': 0.6000000000000001, 'temperature': 2.5}


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at google/bert_uncased_L-2_H-128_A-2 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,3.965500,3.840830,0.280477,0.199898,0.207449,0.157816
2,3.824700,3.710131,0.408799,0.198401,0.308985,0.233811
3,3.703000,3.549631,0.431714,0.373389,0.328452,0.256811
4,3.536100,3.372205,0.518790,0.534913,0.413358,0.378027
5,3.364200,3.200775,0.583868,0.556214,0.480675,0.475059
6,3.203200,3.055242,0.659945,0.604067,0.558012,0.559253
7,3.062400,2.929659,0.689276,0.615554,0.586682,0.585862
8,2.945200,2.818455,0.703025,0.621896,0.599454,0.599792
9,2.832400,2.732872,0.721357,0.630350,0.617804,0.614040
10,2.756200,2.661854,0.726856,0.630142,0.622087,0.618734


[I 2025-03-27 21:53:17,017] Trial 0 pruned. 


...

[I 2025-03-28 00:40:13,126] Trial 148 pruned. 


Trial 149 with params: {'learning_rate': 0.00046921616487852295, 'weight_decay': 0.001, 'warmup_steps': 3, 'lambda_param': 0.0, 'temperature': 4.5}


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at google/bert_uncased_L-2_H-128_A-2 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,3.575300,2.816762,0.694775,0.617780,0.587065,0.590773
2,2.238700,1.627392,0.832264,0.700188,0.711893,0.705600
3,1.233000,1.080120,0.868928,0.730453,0.742120,0.735939
4,0.713600,0.864935,0.876260,0.738005,0.746998,0.742300
5,0.462500,0.776164,0.883593,0.741146,0.753807,0.747151
6,0.323200,0.775091,0.881760,0.904239,0.772023,0.779579
7,0.240700,0.763121,0.887259,0.881195,0.802578,0.820225
8,0.191300,0.760994,0.887259,0.894084,0.829117,0.850330
9,0.157000,0.764362,0.890009,0.891671,0.832775,0.851039
10,0.144400,0.743787,0.891842,0.895856,0.832768,0.853309


[I 2025-03-28 00:41:39,765] Trial 149 finished with value: 0.8565205530377753 and parameters: {'learning_rate': 0.00046921616487852295, 'weight_decay': 0.001, 'warmup_steps': 3, 'lambda_param': 0.0, 'temperature': 4.5}. Best is trial 56 with value: 0.8639964697109707.


In [22]:
print(best_trial2)

BestRun(run_id='56', objective=0.8639964697109707, hyperparameters={'learning_rate': 0.00043703488150776966, 'weight_decay': 0.006, 'warmup_steps': 3, 'lambda_param': 0.7000000000000001, 'temperature': 2.0}, run_summary=None)


In [23]:
#Nápočet epoch na steps
data_length = len(train_aug)
min_r = math.ceil(data_length/batch_size)*5
max_r = math.ceil(data_length/batch_size)*num_epochs
warm_up = math.ceil(data_length/batch_size/10)

In [24]:
base.reset_seed()

In [25]:
training_args = base.get_training_args(output_dir=f"~/results/{DATASET}/bert-base_coarse_aug_hp-search", logging_dir=f"~/logs/{DATASET}/bert-base_coarse_aug_hp-search", epochs=num_epochs, batch_size=batch_size)

In [26]:
def hp_space(trial):
    params =  {
        "learning_rate": trial.suggest_float("learning_rate", 1e-5, 5e-4, log=True),
        "weight_decay": trial.suggest_float("weight_decay", 0, 1e-2, step=1e-3),
        "warmup_steps" : trial.suggest_int("warmup_steps", 0, warm_up),
    }
    print(f"Trial {trial.number} with params: {params}")
    return params

In [27]:
pruner = optuna.pruners.HyperbandPruner(min_resource=min_r, max_resource=max_r, reduction_factor=2, bootstrap_count=2)
sampler = optuna.samplers.TPESampler(seed=42, multivariate=True)

/usr/local/lib/python3.10/dist-packages/optuna/_experimental.py:31: ExperimentalWarning: Argument ``multivariate`` is an experimental feature. The interface can change in the future.
  warnings.warn(


In [28]:
trainer = Trainer(
    args=training_args,
    train_dataset=train_aug,
    eval_dataset=eval,
    compute_metrics=base.compute_metrics,
    model_init = lambda: get_Bert()
)
  

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at google/bert_uncased_L-2_H-128_A-2 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [29]:
best_trial3 = trainer.hyperparameter_search(
    direction="maximize",
    backend="optuna",
    hp_space=hp_space,
    compute_objective=lambda metrics: metrics["eval_f1"],
    pruner=pruner,
    sampler=sampler,
    study_name="Test-base-aug",
    n_trials=150
)

[I 2025-03-28 00:41:40,866] A new study created in memory with name: Test-base-aug


Trial 0 with params: {'learning_rate': 4.3284502212938785e-05, 'weight_decay': 0.01, 'warmup_steps': 23}


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at google/bert_uncased_L-2_H-128_A-2 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,1.347700,0.864301,0.774519,0.664628,0.659635,0.658890
2,0.626700,0.542831,0.852429,0.714985,0.729179,0.721744
3,0.381600,0.452832,0.869844,0.894396,0.770352,0.784002
4,0.279200,0.424973,0.883593,0.862096,0.826964,0.839647
5,0.221700,0.417501,0.874427,0.867224,0.820224,0.836156


[I 2025-03-28 00:43:12,467] Trial 0 pruned. 


...

Trial 148 with params: {'learning_rate': 2.3915063786644546e-05, 'weight_decay': 0.003, 'warmup_steps': 25}


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at google/bert_uncased_L-2_H-128_A-2 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,1.549100,1.220504,0.665445,0.611739,0.557549,0.566460
2,0.989400,0.788955,0.805683,0.681039,0.689293,0.684220
3,0.653900,0.600199,0.842346,0.707469,0.720439,0.713660
4,0.485700,0.515065,0.855179,0.717663,0.731256,0.724118
5,0.391700,0.469631,0.863428,0.723528,0.738051,0.730546
6,0.330500,0.440340,0.873511,0.897731,0.782061,0.800280
7,0.290900,0.426630,0.883593,0.869565,0.817443,0.833917


[W 2025-03-28 07:11:08,404] Trial 148 failed with parameters: {'learning_rate': 2.3915063786644546e-05, 'weight_decay': 0.003, 'warmup_steps': 25} because of the following error: KeyboardInterrupt().
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/optuna/study/_optimize.py", line 197, in _run_trial
    value_or_values = func(trial)
  File "/usr/local/lib/python3.10/dist-packages/transformers/integrations/integration_utils.py", line 250, in _objective
    trainer.train(resume_from_checkpoint=checkpoint, trial=trial)
  File "/usr/local/lib/python3.10/dist-packages/transformers/trainer.py", line 2241, in train
    return inner_training_loop(
  File "/usr/local/lib/python3.10/dist-packages/transformers/trainer.py", line 2581, in _inner_training_loop
    _grad_norm = self.accelerator.clip_grad_norm_(
  File "/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py", line 2510, in clip_grad_norm_
    return torch.nn.utils.clip_grad_norm_(parameters

KeyboardInterrupt: 

In [30]:
print(best_trial3)

NameError: name 'best_trial3' is not defined

In [13]:
base.reset_seed()

In [14]:
training_args = base.get_training_args(output_dir=f"~/results/{DATASET}/bert-distill_coarse_aug_hp-search", logging_dir=f"~/logs/{DATASET}/bert-distill_coarse_aug_hp-search", remove_unused_columns=False, epochs=num_epochs, batch_size=batch_size)

In [15]:
def hp_space(trial):
    params =  {
        "learning_rate": trial.suggest_float("learning_rate", 1e-5, 5e-4, log=True),
        "weight_decay": trial.suggest_float("weight_decay", 0, 1e-2, step=1e-3),
        "warmup_steps" : trial.suggest_int("warmup_steps", 0, warm_up),
        "lambda_param": trial.suggest_float("lambda_param",0,1,step=.1),
        "temperature": trial.suggest_float("temperature", 2,7, step=.5)
    }
    print(f"Trial {trial.number} with params: {params}")
    return params

In [16]:
pruner = optuna.pruners.HyperbandPruner(min_resource=min_r, max_resource=max_r, reduction_factor=2, bootstrap_count=2)
sampler = optuna.samplers.TPESampler(seed=42, multivariate=True)

/usr/local/lib/python3.10/dist-packages/optuna/_experimental.py:31: ExperimentalWarning: Argument ``multivariate`` is an experimental feature. The interface can change in the future.
  warnings.warn(


In [17]:
trainer = base.DistilTrainer(
    args=training_args,
    train_dataset=train_aug,
    eval_dataset=eval,
    compute_metrics=base.compute_metrics,
    model_init = lambda: get_Bert()
)
  

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at google/bert_uncased_L-2_H-128_A-2 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [18]:
best_trial4 = trainer.hyperparameter_search(
    direction="maximize",
    backend="optuna",
    hp_space=hp_space,
    compute_objective=lambda metrics: metrics["eval_f1"],
    pruner=pruner,
    sampler=sampler,
    study_name="Test-Distill-aug",
    n_trials=150
)

[I 2025-03-28 11:31:23,503] A new study created in memory with name: Test-Distill-aug


Trial 0 with params: {'learning_rate': 4.3284502212938785e-05, 'weight_decay': 0.01, 'warmup_steps': 3, 'lambda_param': 0.6000000000000001, 'temperature': 2.5}


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at google/bert_uncased_L-2_H-128_A-2 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,3.056400,2.337134,0.789184,0.674431,0.672664,0.671305


[I 2025-03-28 11:31:42,739] Trial 0 pruned. 


...

[I 2025-03-28 15:44:38,292] Trial 148 pruned. 


Trial 149 with params: {'learning_rate': 0.0001810064352145765, 'weight_decay': 0.008, 'warmup_steps': 2, 'lambda_param': 0.8, 'temperature': 3.5}


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at google/bert_uncased_L-2_H-128_A-2 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,1.724700,0.889067,0.866178,0.729022,0.739529,0.733661
2,0.370500,0.741424,0.886343,0.889197,0.830762,0.848633
3,0.217900,0.656731,0.896425,0.898804,0.846176,0.864210
4,0.173200,0.695757,0.889093,0.882016,0.839776,0.855508
5,0.142300,0.675283,0.889093,0.881389,0.839089,0.854880


[I 2025-03-28 15:47:14,523] Trial 149 pruned. 


In [19]:
print(best_trial4)

BestRun(run_id='122', objective=0.8428319571162907, hyperparameters={'learning_rate': 0.0002362398781984195, 'weight_decay': 0.01, 'warmup_steps': 2, 'lambda_param': 0.7000000000000001, 'temperature': 2.5}, run_summary=None)


In [20]:
print("Best normal training score: ", best_trial)
print("Best distilation trianing score: ", best_trial2)
print("Best normal training score with augmentations: ", best_trial3)
print("Best distilation trianing score with augmentations: ",best_trial4)